In [36]:
import os
import pandas as pd
import numpy as np
from json import load as json_load
from collections import Counter

USER = "Sara".upper()

assert len(list(filter(lambda x: x.startswith("results_") and x.endswith(".json"), os.listdir(f'./results/{USER}/')))) > 0, f"No results files found in './results/{USER}/' directory."
maxR = max([int(fname.split('_')[1].split('.')[0]) for fname in os.listdir(f'./results/{USER}/') if fname.startswith("results_") and fname.endswith(".json")])


path = f'./results/{USER}/results_{maxR}.json'

In [37]:
with open(path, "r", encoding='utf-8') as f:
    results = json_load(f)

assert len(results) > 0, f"No results found in '{path}'."

In [38]:
def computeStats(values:list[dict[str, float]]) -> dict[str, float]:
    return  {
        key: {
            'mean': np.mean([v[key] for v in values]),
            'std': np.std([v[key] for v in values]),
            'min': np.min([v[key] for v in values]),
            'max': np.max([v[key] for v in values]),
        } for key in values[0].keys() if isinstance(values[0][key], float)
    }

In [39]:
classCount = Counter(map(lambda x: x.split("_")[1], results.keys()))

In [40]:
statsPerClass = {
    cls: {
        'count': count,
        **computeStats([
            results[file]['metrics'][0] for file in results.keys() if file.split("_")[1] == cls
        ])
    } for cls, count in classCount.items()
}
statsPerClass

{'NAB': {'count': 28,
  'AUC-PR': {'mean': 0.14983980850861714,
   'std': 0.06585357195756329,
   'min': 0.07336176112346937,
   'max': 0.36366719783867385},
  'AUC-ROC': {'mean': 0.5078212964267966,
   'std': 0.06402437626001267,
   'min': 0.35702636379376906,
   'max': 0.6353270767805747},
  'VUS-PR': {'mean': 0.16831122861644734,
   'std': 0.07546273564669677,
   'min': 0.0753083543487918,
   'max': 0.37413708053569067},
  'VUS-ROC': {'mean': 0.5615599952392085,
   'std': 0.07350128551878811,
   'min': 0.3810886879927592,
   'max': 0.7358121575223852},
  'Standard-F1': {'mean': 0.12910904523829508,
   'std': 0.055719563022459236,
   'min': 0.02557544757033248,
   'max': 0.2899702085402185},
  'PA-F1': {'mean': 0.6665815668778835,
   'std': 0.11406347923105978,
   'min': 0.463699494949495,
   'max': 0.9316338354577057},
  'Event-based-F1': {'mean': 0.22503178427037063,
   'std': 0.08287374590975148,
   'min': 0.040983606557377004,
   'max': 0.43004418262150185},
  'R-based-F1': {'mea

In [41]:
finalStats = computeStats([results[file]['metrics'][0] for file in results.keys()])
finalStats

{'AUC-PR': {'mean': 0.3259504902519509,
  'std': 0.3752198096659314,
  'min': 5.4057560490410185e-06,
  'max': 1.0},
 'AUC-ROC': {'mean': 0.6984320122350054,
  'std': 0.21863542525192933,
  'min': 0.050759702174169585,
  'max': 1.0},
 'VUS-PR': {'mean': 0.3052251778056814,
  'std': 0.349799547360458,
  'min': 2.3309679199814223e-05,
  'max': 1.0},
 'VUS-ROC': {'mean': 0.7785159172490441,
  'std': 0.196829390036117,
  'min': 0.21559791699509245,
  'max': 1.0},
 'Standard-F1': {'mean': 0.08706764688896428,
  'std': 0.13871956078181547,
  'min': 0.0,
  'max': 0.8914549653579676},
 'PA-F1': {'mean': 0.19224575185828663,
  'std': 0.2405406780852835,
  'min': 0.0,
  'max': 1.0},
 'Event-based-F1': {'mean': 0.10558157632742338,
  'std': 0.16526774033754887,
  'min': 0.0,
  'max': 0.9999999999999996},
 'R-based-F1': {'mean': 0.09744153212648556,
  'std': 0.15483540797962583,
  'min': 0.0,
  'max': 0.8973988555799248},
 'accuracy': {'mean': 0.8303341983633011,
  'std': 0.08605749519728566,
  'm

In [42]:
dataset_categories = {
    "UCR": "P&Seq",
    "NAB": "Seq",
    "YAHOO": "P&Seq",
    "IOPS": "Seq",
    "MGAB": "Seq",
    "WSD": "Seq",
    "SED": "Seq",
    "TODS": "P&Seq",
    "NEK": "P&Seq",
    "Stock": "P&Seq",
    "Power": "Seq",
    "Daphnet (U)": "Seq",
    "CATSv2 (U)": "Seq",
    "SWaT (U)": "Seq",
    "LTDB (U)": "Seq",
    "TAO (U)": "P&Seq",
    "Exathlon (U)": "Seq",
    "MITDB (U)": "Seq",
    "MSL (U)": "Seq",
    "SMAP (U)": "Seq",
    "SMD (U)": "Seq",
    "SVDB (U)": "Seq",
    "OPP (U)": "Seq",
}

dataset_categories = {val:{key for key in dataset_categories if dataset_categories[key] == val} for val in set(dataset_categories.values())}

In [43]:
stats = {}
for key in dataset_categories.keys():
    data2 = []
    for file in results.keys():
        if file.split("_")[1] in dataset_categories[key]:
            data2.append(results[file]['metrics'][0])
    stats[key] = computeStats(data2)

pd.DataFrame(stats).map(lambda x: f"{x['mean']:.4f} ± {x['std']:.4f}")

,Seq,P&Seq
AUC-PR,0.1861 ± 0.1672,0.4310 ± 0.4298
AUC-ROC,0.6264 ± 0.1404,0.7681 ± 0.2292
VUS-PR,0.1402 ± 0.1378,0.4104 ± 0.3963
VUS-ROC,0.7860 ± 0.1539,0.8330 ± 0.1912
Standard-F1,0.0506 ± 0.0531,0.0948 ± 0.1656
PA-F1,0.1833 ± 0.2401,0.1236 ± 0.1848
Event-based-F1,0.0687 ± 0.0862,0.0978 ± 0.1700
R-based-F1,0.0394 ± 0.0501,0.1128 ± 0.1823
accuracy,0.8404 ± 0.0602,0.8356 ± 0.0928
precision,0.0379 ± 0.0505,0.0677 ± 0.1567
